# 全过程物理模拟与折射率公式

In [ ]:
import numpy as np
from math import cos, sin, asin, acos, tan, atan, pi, sqrt, exp
import cmath
import matplotlib.pyplot as plt

class Light:
    c = 29979245800  # 光速
    def __init__(self, k, amplitude, phase, phi):
        self.k = k # 波数
        self.wavelength = 1 / self.k # 波长
        self.frequency = self.c / self.wavelength # 频率
        
        self.amplitude = amplitude # 振幅
        self.phi = phi # 参考相角
        self.s = self.amplitude * cos(self.phi) # 垂直于参考平面的分量
        self.phase_s = phase # 垂直于参考平面的相位
        self.p = self.amplitude * sin(self.phi) # 平行参考平面的分量
        self.phase_p = phase # 平行参考平面的相位
    
    def reset(self):
        if self.s < 0:
            self.s = -self.s
            self.phase_s = self.phase_s + pi if self.phase_s < pi else self.phase_s - pi
        if self.p < 0:
            self.p = -self.p
            self.phase_p = self.phase_p + pi if self.phase_p < pi else self.phase_p - pi
        self.phi = atan(self.p / self.s) if self.s != 0 else pi / 2
        # self.amplitude = abs(self.s*cmath.exp(1j*self.phase_s) + self.p*cmath.exp(1j*self.phase_p))
        self.amplitude = sqrt(self.s**2 + self.p**2)

    def spread(self, distance, n):
        dleta_phase = 2 * pi * self.k * distance * n
        self.phase_s += dleta_phase
        self.phase_p += dleta_phase
        while self.phase_s > 2 * pi:
            self.phase_s -= 2 * pi
        while self.phase_p < 0:
            self.phase_p += 2 * pi

    def __str__(self):
        return f"Light(k={self.k}, amplitude={self.amplitude:.6f}, E_s={self.s:.6f}, E_p={self.p:.6f})"
    
    def __repr__(self) -> str:
        return f"Light(k={self.k}, amplitude={self.amplitude:.6f}, E_s={self.s:.6f}, E_p={self.p:.6f})"

In [ ]:
def n_e_SiC(v, epsilon_inf = 6.52, w_LO = 968.0, w_TO = 793.0, GAMMA = 4.76, w_p = 6.0, gamma = 30.0):
    return (cmath.sqrt(epsilon_inf * (1 + (w_LO**2 - w_TO**2) / (w_TO**2 - v**2 - v * gamma * 1j)) - w_p**2 / (v*(v + GAMMA * 1j)))).real

def n_s_SiC(v, epsilon_inf = 6.52, w_LO = 968.0, w_TO = 793.0, GAMMA = 4.76, w_p = 450.0, gamma = 580.0):
    return (cmath.sqrt(epsilon_inf * (1 + (w_LO**2 - w_TO**2) / (w_TO**2 - v**2 - v * gamma * 1j)) - w_p**2 / (v*(v + GAMMA * 1j)))).real

def n_e_Si(v):
    v = v*1e-4
    return 3.41983 + 0.159906*v**2/(1-0.028*v**2) - 0.123109*(v**2/(1-0.028*v**2))**2

def n_s_Si(v):
    return 2

In [ ]:
def Fresnel(theta, n1, n2):
    """Fresnel公式, theta: 入射角, n1: 入射介质折射率, n2: 折射介质折射率"""
    theta = theta * pi / 180 # 入射角
    phi = asin(n1 * sin(theta) / n2) # 折射角
    rs = (n1 * cos(theta) - n2 * cos(phi)) / (n1 * cos(theta) + n2 * cos(phi)) # 垂直于参考平面的分量反射系数
    rp = (n2 * cos(theta) - n1 * cos(phi)) / (n2 * cos(theta) + n1 * cos(phi)) # 平行参考平面的分量反射系数
    ts = 2 * n1 * cos(theta) / (n1 * cos(theta) + n2 * cos(phi)) # 垂直于参考平面的分量透射系数
    tp = 2 * n1 * cos(theta) / (n2 * cos(theta) + n1 * cos(phi)) # 平行参考平面的分量透射系数
    phi = phi * 180 / pi # 折射角
    return rs, rp, ts, tp, phi

In [ ]:
def reflect_(E1: Light, theta, n1, n2, sigma1 = 200*1e-8, sigma2 = 1200*1e-8):
    """修正反射公式, E1: 入射光, theta: 入射角, n1: 入射介质折射率, n2: 折射介质折射率"""
    phi = asin(n1 * sin(theta) / n2) # 折射角
    E2 = Light(E1.k, 1, 0, 0) # 反射光
    E2.phase_s = E1.phase_s
    E2.phase_p = E1.phase_p
    eta1 = exp(-2*(2*pi*n1*cos(theta)*sigma1*E1.k)**2)
    eta2 = exp(-2*(2*pi*n1*cos(theta)*sigma2*E1.k)**2)
    rs = (n1 * cos(theta) - n2 * cos(phi)) / (n1 * cos(theta) + n2 * cos(phi)) # 垂直于参考平面的分量透射系数
    rp = (n2 * cos(theta) - n1 * cos(phi)) / (n2 * cos(theta) + n1 * cos(phi)) # 平行参考平面的分量透射系数
    if n1 == 1:
        E2.s = E1.s * rs * eta1
        E2.p = E1.p * rp * eta1
    else:
        E2.s = E1.s * rs * eta2
        E2.p = E1.p * rp * eta2
    E2.reset()
    return E2

In [ ]:
def refract(E1: Light, theta, n1, n2):
    """折射公式, E1: 入射光, theta: 入射角, n1: 入射介质折射率, n2: 折射介质折射率"""
    phi = asin(n1 * sin(theta) / n2) # 折射角
    E2 = Light(E1.k, 1, 0, 0) # 折射光
    E2.phase_s = E1.phase_s
    E2.phase_p = E1.phase_p
    ts = 2 * n1 * cos(theta) / (n1 * cos(theta) + n2 * cos(phi)) # 垂直于参考平面的分量透射系数
    tp = 2 * n1 * cos(theta) / (n2 * cos(theta) + n1 * cos(phi)) # 平行参考平面的分量透射系数
    E2.s = E1.s * ts
    E2.p = E1.p * tp
    E2.reset()
    return E2, phi

In [ ]:
def interference_mul(E_list):
    """干涉公式, E_list: 光的列表"""
    E3 = Light(E_list[0].k, 1, 0, 0) # 干涉光
    E3.phase_s = E_list[0].phase_s
    E3.phase_p = E_list[0].phase_p
    E3.s = E_list[0].s
    E3.p = E_list[0].p
    for E in E_list[1:]:
        E3.s = sqrt(E3.s**2 + E.s**2 + 2 * E3.s * E.s * cos(E3.phase_s - E.phase_s))
        E3.p = sqrt(E3.p**2 + E.p**2 + 2 * E3.p * E.p * cos(E3.phase_p - E.phase_p))
        E3.reset()
    return E3

In [ ]:
def R_(k, d, theta = 10, sigma1 = 200*1e-8, sigma2 = 1200*1e-8, n_e = n_e_SiC, n_s = n_s_SiC, TEST = False):
    """双光束干涉总反射模拟, k: 波数(cm^-1), d: 厚度(um), theta(°): 入射角"""
    theta = theta * pi / 180 # 入射角
    d = d * 1e-4 # 厚度
    E0 = Light(k, 1, 0, pi/4) # 入射光
    if TEST:
        print("入射光",E0)
    n0 = 1 # 空气折射率
    ne = n_e(k) # 外延折射率
    if TEST:
        print("折射率",ne)
    ns = n_s(k) # 衬底折射率
    E1 = reflect_(E0, theta, n0, ne, sigma1, sigma2) # 反射光
    if TEST:
        print("一次反射光",E1)
    E1_, phi = refract(E0, theta, n0, ne) # 透射光
    E1.spread(2*d*tan(phi)*sin(theta), n0) # 反射光传播
    E1_.spread(d*cos(phi), ne) # 透射光传播
    E1__ = reflect_(E1_, phi, ne, ns, sigma1, sigma2) # 透射光反射
    E1__.spread(d*cos(phi), ne) # 透射光传播
    E2, _ = refract(E1__, phi, ne, n0) # 透射光出射
    if TEST:
        print("二次反射光",E2)

    ER = interference_mul([E1, E2]) # 总反射光
    if TEST:
        print("总反射光",ER)
    return 100 * ER.amplitude ** 2

In [ ]:
def R_mul_(k, d, theta = 10, sigma1 = 200*1e-8, sigma2 = 1200*1e-8, n_e = n_e_SiC, n_s = n_s_SiC, TEST = False):
    """多光束干涉总反射模拟, k: 波数(cm^-1), d: 厚度(um), theta(°): 入射角"""
    theta = theta * pi / 180 # 入射角
    d = d * 1e-4 # 厚度
    E0 = Light(k, 1, 0, pi/4) # 入射光
    if TEST:
        print("入射光",E0)
    n0 = 1 # 空气折射率
    ne = n_e(k) # 外延折射率
    if TEST:
        print("折射率",ne)
    ns = n_s(k) # 衬底折射率
    E1 = reflect_(E0, theta, n0, ne, sigma1, sigma2) # 反射光
    if TEST:
        print("一次反射光",E1)
    E1_, phi = refract(E0, theta, n0, ne) # 透射光
    E1.spread(2*d*tan(phi)*sin(theta), n0) # 反射光传播
    E1_.spread(d*cos(phi), ne) # 透射光传播
    E1__ = reflect_(E1_, phi, ne, ns, sigma1, sigma2) # 透射光反射
    E1__.spread(d*cos(phi), ne) # 透射光传播
    E2, _ = refract(E1__, phi, ne, n0) # 透射光出射
    if TEST:
        print("二次反射光",E2)
    E2_ = reflect_(E1__, phi, ne, n0, sigma1, sigma2) # 反射光反射
    E2_.spread(d*cos(phi), ne) # 透射光传播
    E2__= reflect_(E2_, phi, ne, ns, sigma1, sigma2) # 透射光反射
    E2__.spread(d*cos(phi), ne) # 透射光传播
    E3, _ = refract(E2__, phi, ne, n0) # 透射光出射
    if TEST:
        print("三次反射光",E3)
    E3_ = reflect_(E2__, phi, ne, n0, sigma1, sigma2) # 反射光反射
    E3_.spread(d*cos(phi), ne) # 透射光传播
    E3__= reflect_(E3_, phi, ne, ns, sigma1, sigma2) # 透射光反射
    E3__.spread(d*cos(phi), ne) # 透射光传播
    E4, _ = refract(E3__, phi, ne, n0) # 透射光出射
    if TEST:
        print("四次反射光",E4)
    E4_ = reflect_(E3__, phi, ne, n0, sigma1, sigma2) # 反射光反射
    E4_.spread(d*cos(phi), ne) # 透射光传播
    E4__= reflect_(E4_, phi, ne, ns, sigma1, sigma2) # 透射光反射
    E4__.spread(d*cos(phi), ne) # 透射光传播
    E5, _ = refract(E4__, phi, ne, n0) # 透射光出射
    if TEST:
        print("五次反射光",E5)

    ER = interference_mul([E1, E2, E3, E4, E5]) # 总反射光
    if TEST:
        print("总反射光",ER)
    return 100 * ER.amplitude ** 2

In [ ]:
def Delta_phi(v, theta, n_e = n_e_SiC, n_s = n_s_SiC):
    """附加相位公式"""
    n0 = 1
    ne = n_e(v)
    #print(ne)
    ns = n_s(v)
    rs1, rp1, ts1, tp1, phi = Fresnel(theta, n0, ne)
    #print(phi)
    rs2, rp2, ts2, tp2, _ = Fresnel(phi, ne, ns)
    rs1_, rp1_, ts1_, tp1_, theta = Fresnel(phi, ne, n0)
    #print(theta)
    return - atan(rp1/rs1) + atan(tp1_*rp2*tp1/(ts1_*rs2*ts1))

# 最小二乘拟合

In [ ]:
def R_(k, d, theta = 10.0, sigma1 = 2*1e-6, sigma2 = 12*1e-6,
       epsilon_inf_e = 6.862, w_LO_e = 971.4, w_TO_e = 796.2, GAMMA_e = 4.333, w_p_e = 75.79, gamma_e = 180.0,
       epsilon_inf_s = 6.776, w_LO_s = 959.1, w_TO_s = 791.8, GAMMA_s = 4.21, w_p_s = 545.0, gamma_s = 554.6,
       n_e = n_e_SiC, n_s = n_s_SiC):
    """双光束干涉总反射率公式, k: 波数(cm^-1), d: 厚度(um), theta(°): 入射角"""
    theta = theta * pi / 180 # 入射角
    d = d * 1e-4 # 厚度
    n0 = 1 # 空气折射率
    ne = n_e(k, epsilon_inf_e, w_LO_e, w_TO_e, GAMMA_e, w_p_e, gamma_e) # 外延折射率
    ns = n_s(k, epsilon_inf_s, w_LO_s, w_TO_s, GAMMA_s, w_p_s, gamma_s) # 衬底折射率
    phi = asin(sin(theta) / ne) # 透射角
    r1 = (n0 - ne) / (n0 + ne) # 界面1近正入射反射
    t1 = 2*n0 / (n0 + ne) # 界面1近正入射透射
    r2 = (ne - ns) / (ne + ns) # 界面2近正入射反射
    t1_ = 2*ne / (n0 + ne) # 界面1反近正入射透射
    delta = 4*pi*k*ne*d*cos(phi) + pi - Delta_phi(k, theta)
    eta1 = exp(-2*(2*pi*n0*cos(theta)*sigma1*k)**2)
    eta2 = exp(-2*(2*pi*ne*cos(phi)*sigma2*k)**2)
    return 100 * (abs(eta1*r1+eta2*t1*t1_*r2*cmath.exp(1j*delta)))**2

In [ ]:
def R_mul_(k, d, theta = 10.0, sigma1 = 2*1e-6, sigma2 = 12*1e-6,
       epsilon_inf_e = 6.862, w_LO_e = 971.4, w_TO_e = 796.2, GAMMA_e = 4.333, w_p_e = 75.79, gamma_e = 180.0,
       epsilon_inf_s = 6.776, w_LO_s = 959.1, w_TO_s = 791.8, GAMMA_s = 4.21, w_p_s = 545.0, gamma_s = 554.6,
       n_e = n_e_SiC, n_s = n_s_SiC):
    """总反射率公式, k: 波数(cm^-1), d: 厚度(um), theta(°): 入射角"""
    theta = theta * pi / 180 # 入射角
    d = d * 1e-4 # 厚度
    n0 = 1 # 空气折射率
    ne = n_e(k, epsilon_inf_e, w_LO_e, w_TO_e, GAMMA_e, w_p_e, gamma_e) # 外延折射率
    ns = n_s(k, epsilon_inf_s, w_LO_s, w_TO_s, GAMMA_s, w_p_s, gamma_s) # 衬底折射率
    phi = asin(sin(theta) / ne) # 透射角
    r1 = (n0 - ne) / (n0 + ne) # 界面1近正入射反射
    t1 = 2*n0 / (n0 + ne) # 界面1近正入射透射
    r2 = (ne - ns) / (ne + ns) # 界面2近正入射反射
    t1_ = 2*ne / (n0 + ne) # 界面1反近正入射透射
    delta = 4*pi*k*ne*d*cos(phi) + pi - Delta_phi(k, theta)
    eta1 = exp(-2*(2*pi*n0*cos(theta)*sigma1*k)**2)
    eta2 = exp(-2*(2*pi*ne*cos(phi)*sigma2*k)**2)
    return 100 * (abs((eta1*r1+eta2*r2*cmath.exp(1j*delta))/(1+eta1*eta2*r1*r2*cmath.exp(1j*delta))))**2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize, dual_annealing
from scipy.optimize import least_squares
from functools import partial

# 设置字体和负号显示
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

import pandas as pd

def fitting(i):
    # 读取Excel文件
    data = pd.read_excel(f'附件{i}.xlsx', header=None)
    data = data.iloc[1:]  # 跳过第一行（列标题）

    # 提取波数和反射率数据
    wavenumber = np.array(data.iloc[:, 0])  # 波数 (cm-1)
    reflectance = np.array(data.iloc[:, 1])  # 反射率 (%)

    # 定义参数边界
    bounds = [(0.0, 20.0)]

    # 初始猜测
    x0 = np.array([10.0])

    sigma1 = 2*1e-6
    sigma2 = 12*1e-6

    def obj_(X:np.ndarray):
        theormo = np.array([R_(wavenumber[i], X[0], 10, sigma1, sigma2) for i in range(len(wavenumber))])
        return np.sum((reflectance - theormo) ** 2) * 1e-4

    # 使用Nelder-Mead算法
    result_nm = minimize(obj_, x0, method='Nelder-Mead', bounds=bounds)
    print(f"最优参数: {result_nm.x}")
    print(f"最小值: {result_nm.fun}")
    print(f"是否成功: {result_nm.success}")
    print(f"消息: {result_nm.message}\n")

    best_method = 'Nelder-Mead'
    best_result = result_nm
    best_params = best_result.x if hasattr(best_result, 'x') else best_result.x
    best_value = result_nm.fun

    # 绘制拟合曲线
    best_theormo = np.array([R_(wavenumber[i], best_params[0], 10, sigma1, sigma2) for i in range(len(wavenumber))])

    # 创建图表
    plt.figure(figsize=(12, 8))

    # 绘制原始数据
    plt.plot(wavenumber, reflectance, 'bo', label='实际反射率', markersize=4)

    # 绘制拟合曲线
    plt.plot(wavenumber, best_theormo, 'r-', label=f'拟合反射率 ({best_method})', linewidth=2)

    # 添加图表标题和坐标轴标签
    plt.title('反射率拟合曲线', fontsize=16)
    plt.xlabel('波数 (cm$^{-1}$)', fontsize=12)
    plt.ylabel('反射率', fontsize=12)

    # 添加网格和图例
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend()

    # 优化布局
    plt.tight_layout()

    # 显示图表
    plt.show()

    return best_params[0]

In [ ]:
def fitting_mul(i):
    # 读取Excel文件
    data = pd.read_excel(f'附件{i}.xlsx', header=None)
    data = data.iloc[1:]  # 跳过第一行（列标题）

    # 提取波数和反射率数据
    wavenumber = np.array(data.iloc[:, 0])  # 波数 (cm-1)
    reflectance = np.array(data.iloc[:, 1])  # 反射率 (%)

    # 定义参数边界
    bounds = [(0.0, 20.0), (0.0, 10), (1.0, 20), 
            (0.0, 10), (500, 2000), (100, 1000), (0.0, 10), (0.0, 10), (0.0, 100), 
            (0.0, 10), (500, 2000), (100, 1000), (0.0, 10), (200, 1000), (200, 1000)]

    # 初始猜测
    x0 = np.array([10.0, 2, 12, 6.52, 968, 793, 4.76, 6, 30, 6.52, 968, 793, 4.76, 450, 580])

    sigma1 = 2*1e-6
    sigma2 = 12*1e-6
    ns = 2.25

    def obj_(X:np.ndarray):
        theormo = np.array([R_(wavenumber[i], X[0], 10, X[1]*1e-6, X[2]*1e-6,
                            X[3], X[4], X[5], X[6], X[7], X[8],
                            X[9], X[10], X[11], X[12], X[13], X[14]) for i in range(len(wavenumber))])
        return np.sum((reflectance - theormo) ** 2) * 1e-4

    # 使用Nelder-Mead算法
    result_nm = minimize(obj_, x0, method='Nelder-Mead', bounds=bounds)
    print(f"最优参数: {result_nm.x}")
    print(f"最小值: {result_nm.fun}")
    print(f"是否成功: {result_nm.success}")
    print(f"消息: {result_nm.message}\n")

    best_method = 'Nelder-Mead'
    best_result = result_nm
    best_params = best_result.x if hasattr(best_result, 'x') else best_result.x
    best_value = result_nm.fun

    # 绘制拟合曲线
    best_theormo = np.array([R_(wavenumber[i], best_params[0], 10, best_params[1]*1e-6, best_params[2]*1e-6,
                                best_params[3], best_params[4], best_params[5], best_params[6], best_params[7], best_params[8], 
                                best_params[9], best_params[10], best_params[11], best_params[12], best_params[13], best_params[14]) for i in range(len(wavenumber))])

    # 创建图表
    plt.figure(figsize=(12, 8))

    # 绘制原始数据
    plt.plot(wavenumber, reflectance, 'bo', label='实际反射率', markersize=4)

    # 绘制拟合曲线
    plt.plot(wavenumber, best_theormo, 'r-', label=f'拟合反射率 ({best_method})', linewidth=2)

    # 添加图表标题和坐标轴标签
    plt.title('反射率拟合曲线', fontsize=16)
    plt.xlabel('波数 (cm$^{-1}$)', fontsize=12)
    plt.ylabel('反射率', fontsize=12)

    # 添加网格和图例
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend()

    # 优化布局
    plt.tight_layout()

    # 显示图表
    plt.show()

# 基于极值的统计分析

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from scipy import stats

# 设置字体和负号显示
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

def moving_average(data, window_size=5):
    """
    移动平均滤波
    
    参数:
    data -- 输入数据
    window_size -- 窗口大小，必须是奇数
    
    返回:
    滤波后的数据
    """
    if window_size % 2 == 0:
        window_size += 1  # 确保窗口大小为奇数
    
    # 使用卷积实现移动平均
    window = np.ones(window_size) / window_size
    return np.convolve(data, window, mode='same')

def get_d(i, n_e = n_e_SiC):
    theta = 10 if i%2==1 else 15

    # 读取Excel文件
    data = pd.read_excel(f'D:\\陈信嘉2025\\数学建模\\附件{i}.xlsx', header=None)
    data = data.iloc[1:]  # 跳过第一行（列标题）

    # 提取波数和反射率数据
    wavenumber = np.array(data.iloc[:, 0])  # 波数 (cm-1)
    reflectance = np.array(data.iloc[:, 1])  # 反射率 (%)

    # 创建图表
    plt.figure(figsize=(14, 10))

    # 绘制原始光谱
    plt.plot(wavenumber, reflectance, 'b-', linewidth=1, alpha=0.5, label='原始光谱')

    # 移动平均滤波
    reflectance = moving_average(reflectance, window_size=35)

    # 使用Scipy的find_peaks函数
    peaks, properties = find_peaks(reflectance, height=0, prominence=0.5, width=1)
    valleys, properties = find_peaks(-reflectance, height=-100, prominence=0.5, width=1)

    # 绘制滤波后的光谱
    plt.plot(wavenumber, reflectance, 'r-', linewidth=1.5, label='移动平均滤波')

    plt.plot(wavenumber[peaks], reflectance[peaks], "v", color='blue', label="Peaks") # 绘制峰值
    plt.plot(wavenumber[valleys], reflectance[valleys], "^", color='green', label="Valleys")  # 绘制谷值

    # 添加图表标题和坐标轴标签
    plt.xlabel('波数 (cm$^{-1}$)', fontsize=12)
    plt.ylabel('反射率 (%)', fontsize=12)

    # 添加网格和图例
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend()

    # 优化布局
    plt.tight_layout()

    # 显示图表
    plt.show()              
    
    kn = wavenumber[peaks][10:15]
    kn2 = wavenumber[valleys][10:15]
    k_n = np.append(kn, kn2)
    k_n.sort()
    print(k_n)
    print(np.diff(k_n))

    theta = theta * pi/180
    lamda = [1/k_n[i] for i in range(len(k_n))]
    A = [sqrt(n_e(k_n[i])**2 - sin(theta)**2) for i in range(len(k_n))]
    PHI = [Delta_phi(k_n[i], theta) for i in range(len(k_n))]
    p = {}
    for i in range(len(k_n)):
        p[i] = [(0.5*(lamda[j]*A[i] - lamda[i]*A[j])+0.5/pi*(PHI[i]*lamda[i]*A[j]-PHI[j]*lamda[j]*A[i])-0.5*(j-i)*lamda[j]*A[i])/(lamda[j]*A[i] - lamda[i]*A[j]) if i != j else 0 for j in range(len(k_n))]
    d = np.zeros((len(k_n), len(k_n)))
    for i in range(len(k_n)):
        for j in range(len(k_n)):
            d[i][j] = 10000*(int(p[i][j]*2)/2 - 0.5 + PHI[i]/2/pi)*lamda[i]/(2*A[i]) if i != j else 0

    # 创建图表
    plt.figure(figsize=(12, 10))

    # 使用热图可视化d矩阵
    # 使用seaborn的heatmap函数，它可以提供更好的可视化效果
    sns.heatmap(d, 
                annot=True,          # 在单元格中显示数值
                fmt=".2f",           # 数值格式
                cmap='viridis',      # 颜色映射
                cbar_kws={'label': 'd值'},  # 颜色条标签
                linewidths=0.5,      # 单元格之间的线宽
                linecolor='white')   # 单元格之间的线颜色

    # 设置图表标题和坐标轴标签
    plt.title('矩阵d的热图', fontsize=16)
    plt.xlabel('j索引', fontsize=12)
    plt.ylabel('i索引', fontsize=12)

    # 优化布局
    plt.tight_layout()

    # 显示图表
    plt.show()

    # 打印矩阵d的一些统计信息
    print("厚度d的统计信息:")
    d_list = []
    for i in range(len(k_n)):
        for j in range(i, len(k_n)):
            if i != j:
                d_list.append(d[i][j])
    del_d = []
    for i in range(len(d_list)):
        if abs(d_list[i] - np.mean(d_list)) > 3*np.std(d_list):
            del_d.append(d_list[i])
    for d0_ in del_d:
        d_list.remove(d0_)
    x = np.arange(len(d_list))
    plt.scatter(x, d_list, label='d值')
    plt.xlabel('样本编号', fontsize=12)
    plt.ylabel('厚度d (nm)', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend()
    plt.show()
    print(f"最小值: {np.min(d_list):.4f}")
    print(f"最大值: {np.max(d_list):.4f}")
    print(f"平均值: {np.mean(d_list):.4f}")
    print(f"标准差: {np.std(d_list):.4f}")
    print(f"中位数: {np.median(d_list):.4f}")

    # 中心趋势
    d_avg = np.mean(d_list)
    print(f"平均值: {d_avg:.4f}")
    # 方差
    d_var = np.var(d_list)
    print(f"方差: {d_var:.4f}")
    # 标准偏差
    d_std = np.std(d_list)
    print(f"标准偏差: {d_std:.4f}")
    # 标准误差
    d_se = d_std / np.sqrt(len(d_list))
    print(f"标准误差: {d_se:.4f}")
    # 95% 置信区间
    d_ci = stats.t.ppf(0.975, len(d_list)-1) * d_se
    print(f"95% 置信区间: ({d_avg-d_ci:.4f}, {d_avg+d_ci:.4f})")
    # 变异系数
    d_cv = d_std / d_avg * 100
    if d_cv < 5:
        print(f"变异系数: {d_cv:.4f}% < 5%, 说明数据具有很好的可靠性")
    elif d_cv < 10:
        print(f"变异系数: {d_cv:.4f}% < 10%, 说明数据具有较好的可靠性")
    else:
        print(f"变异系数: {d_cv:.4f}% >= 10%, 说明数据可靠性一般")
    # 可靠性估计
    reliability = 1 - d_var / (d_avg ** 2)
    print(f"可靠性估计: {reliability:.4f}")

    return d_avg, d_std

# 半宽度判定多光束干涉

In [ ]:
import pandas as pd
import numpy as np
from scipy.signal import argrelextrema
import matplotlib.pyplot as plt

def calculate_half_width(data, y_line=2.5):
    # 1. 读取数据
    data = pd.read_excel('附件3.xlsx', header=0, names=['波数 (cm-1)', '反射率 (%)'])
    x = data['波数 (cm-1)'].values
    y = data['反射率 (%)'].values

    # 2. 寻找极值点
    # 寻找局部极大值 (peak)
    max_indices = argrelextrema(y, np.greater, order=100)[0] # order参数控制灵敏度，可根据数据调整
    # 寻找局部极小值 (valley)
    min_indices = argrelextrema(y, np.less, order=100)[0]

    # 确保极值点按X轴顺序排列
    max_peaks = sorted(zip(x[max_indices], y[max_indices]), key=lambda i: i[0])
    min_valleys = sorted(zip(x[min_indices], y[min_indices]), key=lambda i: i[0])

    # 3. & 4. 为每个极小值寻找半宽度点并计算比值
    results = []

    for i, valley in enumerate(min_valleys):
        v_x, v_y = valley
        
        # 找到当前极小值左右相邻的极大值
        left_peak = None
        right_peak = None
        
        # 寻找左侧极大值 (X坐标小于当前极小值)
        for peak in reversed(max_peaks):
            if peak[0] < v_x:
                left_peak = peak
                break
                
        # 寻找右侧极大值 (X坐标大于当前极小值)
        for peak in max_peaks:
            if peak[0] > v_x:
                right_peak = peak
                break
                
        # 如果找不到任何一侧的极大值，跳过这个极小值
        if left_peak is None or right_peak is None:
            print(f"Warning: Cannot find adjacent peaks for valley at {v_x}. Skipping.")
            continue
            
        l_peak_x, l_peak_y = left_peak
        r_peak_x, r_peak_y = right_peak
        
        # 计算半高度
        half_height = v_y + (min(l_peak_y, r_peak_y) - v_y) / 2
        # 更稳健的做法：使用两侧极大值中较低的那个来计算半高
        # 这适用于非对称峰，确保半高线不会与另一侧的峰相交
        
        # 在当前极小值左右两侧的数据中寻找半高度点
        # 左侧数据 (从left_peak到valley)
        left_region_indices = np.where((x >= l_peak_x) & (x <= v_x))
        left_region_x = x[left_region_indices]
        left_region_y = y[left_region_indices]
        # 插值找到Y值第一次等于half_height的点
        left_half_point_x = np.interp(half_height, left_region_y[::-1], left_region_x[::-1])
        
        # 右侧数据 (从valley到right_peak)
        right_region_indices = np.where((x >= v_x) & (x <= r_peak_x))
        right_region_x = x[right_region_indices]
        right_region_y = y[right_region_indices]
        right_half_point_x = np.interp(half_height, right_region_y, right_region_x)
        
        # 计算半宽度 (FWHM)
        fwhm = right_half_point_x - left_half_point_x
        
        # 计算两侧极大值的X距离
        peak_distance = r_peak_x - l_peak_x
        
        # 计算比值
        ratio = peak_distance / fwhm
        
        # 存储结果
        results.append({
            'Valley_X': v_x,
            'Left_Half_Point_X': left_half_point_x,
            'Right_Half_Point_X': right_half_point_x,
            'FWHM': fwhm,
            'Left_Peak_X': l_peak_x,
            'Right_Peak_X': r_peak_x,
            'Peak_Distance': peak_distance,
            'Ratio': ratio
        })

    # 将结果转换为DataFrame以便分析
    df_results = pd.DataFrame(results)

    # 5. 结果可视化
    # 绘制原始数据及极值点

    plt.figure(figsize=(12, 6))
    plt.plot(x, y, label='Reflectance Spectrum', color='lightgray')
    # 标记极值点
    plt.scatter(x[max_indices], y[max_indices], color='green', marker='^', s=50, label='Maxima (Peaks)')
    plt.scatter(x[min_indices], y[min_indices], color='red', marker='v', s=50, label='Minima (Valleys)')
    # 标记半宽度点和线
    for res in results:
        v_x = res['Valley_X']
        lh_x = res['Left_Half_Point_X']
        rh_x = res['Right_Half_Point_X']
        half_height_val = np.interp(lh_x, x, y) # 获取半高度点的Y值用于绘图
        plt.hlines(y=half_height_val, xmin=lh_x, xmax=rh_x, colors='blue', linestyles='dashed')
        plt.scatter([lh_x, rh_x], [half_height_val, half_height_val], color='blue', marker='o', s=30, label='Half-Width Points' if 'Half-Width Points' not in plt.gca().get_legend_handles_labels()[1] else "")
        plt.text(v_x, res['Valley_X']*0.9, f"Ratio: {res['Ratio']:.2f}", fontsize=8, ha='center')
    plt.xlabel('Wavenumber (cm⁻¹)')
    plt.ylabel('Reflectance (%)')
    plt.title('Reflectance Spectrum with Extrema and Half-Width Analysis')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('极值与半宽度分析.png', dpi=300, bbox_inches='tight')
    plt.show()

    # 绘制比值变化图
    plt.figure(figsize=(10, 5))
    plt.plot(df_results['Valley_X'], df_results['Ratio'], 'o-', color='orange')
    plt.xlabel('Wavenumber of Valley (cm⁻¹)')
    plt.ylabel('Ratio (Peak Distance)')
    plt.title('Change of Ratio (Peak Distance) for Each Minimum')
    plt.grid(True, alpha=0.3)

    # 为每个点标注具体值
    for i, row in df_results.iterrows():
        plt.annotate(f"{row['Ratio']:.2f}", (row['Valley_X'], row['Ratio']), textcoords="offset points", xytext=(0,10), ha='center', fontsize=8)

    # 添加y=2.5的水平直线
    y_line = 2.5
    plt.axhline(y=y_line, color='blue', linestyle='--', label='y=2.5')

    # 查找交点（线性插值法）
    vx = df_results['Valley_X'].values
    vy = df_results['Ratio'].values
    cross_points = []
    for i in range(1, len(vx)):
        if (vy[i-1] - y_line) * (vy[i] - y_line) < 0:
            # 线性插值求交点x
            x_cross = vx[i-1] + (vx[i] - vx[i-1]) * (y_line - vy[i-1]) / (vy[i] - vy[i-1])
            cross_points.append((x_cross, y_line))
            plt.scatter(x_cross, y_line, color='red', zorder=5)
            plt.annotate(
        f"({x_cross:.1f}, {y_line})",
        (x_cross, y_line),
        fontsize=10,
        color='red'
    )
    plt.tight_layout()
    plt.show()